In [ ]:
  from google.colab import drive
drive.mount('/content/drive/')

In [ ]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

In [ ]:
!mkdir -p drive
!google-drive-ocamlfuse drive
!ls

In [ ]:
!ls drive

In [ ]:
import os 
os.chdir("/content/drive/My Drive/MOYERS")
!pwd

In [ ]:
!ls

In [7]:

%matplotlib inline

import pandas as pd
import numpy as np
import tensorflow as tf
import keras

from keras import utils
from keras.models import Sequential , Model,model_from_json ,load_model
from keras.layers import Dense , Dropout , Activation
from keras.datasets import mnist
from keras.preprocessing import image
from keras.callbacks import ModelCheckpoint
from matplotlib import pyplot as plt



In [36]:
veri= pd.read_csv('moyers812.csv')
veri_cols = ['molar', 'santral', 'lateral','grup'] 
#veri.columns = veri_cols
giris_veri=veri.iloc[:,0:3].values
cikis_veri=veri.iloc[:,3].values
veri1= pd.read_csv('moyers812_test.csv')
veri1_cols = ['molar', 'santral', 'lateral','grup'] 
#veri1.columns = veri1_cols
x_test=veri.iloc[:,0:3].values
y_test=veri.iloc[:,3].values



In [37]:
#veri niteliklerini standardize edeceğiz
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
giris_veri = scaler.fit_transform(giris_veri)
x_test = scaler.transform(x_test)

In [38]:
model = Sequential() 

In [39]:
 #Tam bağlantı katmanı

model.add(Dense(512, input_dim = 3  ,activation='relu'))

model.add(Dense(256, activation='relu'))

model.add(Dense(64, activation='relu'))

model.add(Dense(32, activation='relu'))

model.add(Dense(16, activation='relu'))


model.add(Dense(6, activation='softmax'))

model.compile(optimizer= 'adam' , loss='sparse_categorical_crossentropy' , metrics=['accuracy'])


In [ ]:
checkpointer = ModelCheckpoint('mnist_model.h5',verbose=1, save_best_only= True)


hist=model.fit(giris_veri,cikis_veri,
          batch_size=10,
          shuffle= True,
          epochs =200,
          validation_data=(x_test, y_test),
          callbacks = [checkpointer], verbose = 2)
model_json = model.to_json()
with open("mnist_model.json","w") as json_file:
    json_file.write(model_json)


In [ ]:
#
plt.figure(figsize=(14,3))
plt.subplot(1,2,1)
plt.suptitle('eğitim', fontsize =10)
plt.ylabel('Loss', fontsize=16)
plt.plot(hist.history['loss'], color='b',label ='training loss')
plt.plot(hist.history['val_loss'], color='r',label ='validation loss')
plt.legend(loc='upper right')

plt.subplot(1,2,2)
plt.ylabel('Accuracy', fontsize=16)
plt.plot(hist.history['accuracy'], color='b',label ='training accuracy')
plt.plot(hist.history['val_accuracy'], color='r',label ='validation accuracy')
plt.legend(loc='lower right')
plt.show()




In [ ]:
model.summary()

In [15]:
model_best= load_model('mnist_model.h5')
checkpointer = ModelCheckpoint ('mnist_model.h5',verbose=1, save_best_only=True)

In [ ]:
score = model.evaluate(x_test, y_test, verbose=0)
print('Test Loss:', score[0])
print('Test Accuracy:', score[1])


In [ ]:
veri1= pd.read_csv('TEST2.csv')
veri1 = np.array(veri1)
xx_test= veri1[:,0:3]
xy_test=veri1[:,3]
xy_test =  keras.utils.to_categorical(xy_test,num_classes=6)

for j in range(len(xy_test)):
  test_1 = xx_test[j]
  test_1= np.expand_dims(test_1,axis=0)
  test1 =np.vstack([test_1])
  results =model_best.predict(test1, batch_size=1)
  
  print ("doğru sınıf " , xy_test[j])
  print( results)
  objects = ( 'A', 'B', 'C','D' ,'E')
  ind= 0.1+0.6*np.arange(len(objects))
  width = 0.5
  color_list =['red' , 'orangered', 'darkorange','limegreen' , 'darkgreen']

  for i in range(len(objects)):
    plt.bar(ind[i],results[0][i+1], width, align='center', color=color_list[i])
  
  plt.title("sonuçlar")
  plt.xlabel("dönemler")
  plt.ylabel("tahmin skoru")
  plt.xticks(ind, objects,rotation=45 , fontsize=14)
  plt.show()
# print ("tahmin edilen dönem ",objects[np.argmax(results)])


In [ ]:
%matplotlib inline
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import plot_confusion_matrix
from matplotlib import pyplot as plt
plt.figure(figsize=(20, 20))
clf = GradientBoostingClassifier(random_state=42)
clf.fit(giris_veri, cikis_veri)
plot_confusion_matrix(clf, x_test, y_test, cmap=plt.cm.Blues)
plt.title('Confusion matrix')
plt.show()